# Problema 1: Optimización Multi objetivo en Distribución de Recursos para Misión Humanitaria

## 1. Formulación del Modelo Multiobjetivo

### Conjuntos

* Identificador de cada recurso 

$$ I\in \left[Alimentos\:basicos,\:Medicinas,\:Equipos\:medicos,\:Agua\:potable,\:Mantas\right] $$

* Identificador de cada avion disponible

$$ J\in \left[1,\:2,\:3,\:4\right] $$

* Identificador de cada zona de destino 

$$ K\in \left[A,\:B,\:C,\:D\right] $$

* Identificador de viaje por avion 

$$ V\in \left[0,\:1,\:2\right] $$


### Parametros

* Valor de impacto de cada recurso (USD/TON)

$$ VI_i $$

* Peso de cada recurso (TON/unidad)

$$ P_i $$

* Volumen de cada recurso ($m^3$/unidad)

$$ V_i $$

* Diponibilidad de cada recurso (Unidad)

$$ D_i $$

* Peso que puede soportar un avion (TON)

$$ CP_j $$

* Capacidad que tiene un avion ($m^3$)

$$ CV_j $$

* Coste fijo por uso de un avion (USD)

$$ CF_j $$

* Costo variable por el uso de un avion (USD/Km)

$$ CK_j $$

* Distancia de una zona (Km)

$$ DZ_k $$

* Poblacion de una zona (Miles)

$$ N_k $$

* Multiplicador de impacto de una zona 

$$ MI_k $$

* Necesidad minima de un recurso en una zona 

$$ N_{i,k} $$

### Variables de decision 

* Cantidad de recurso $ i $ transportada en el avion $ j $ en el viaje $ v $ a la zona $ k $ 

$$ x_{i,j,v,k}\in Z^+ $$

* Variable binaria que indica si un avion $ j $ hace un viaje $ v $ en una zona $ k $ 

$$ y_{j,v,k}\in \left[0,\:1\right] $$

* Variables binaria que indica si un avion $ j $ es utilizado 

$$ z_j=\left[0,\:1\right] $$

* Variable binaria que indica la presencia de un recurso $ i $ en el avion $ j $ en un viaje $ v $ 

$$ p_{i,j,v}=\left[0,\:1\right] $$

### Funciones objetivo

* Calculo de valor de impacto social 

$$ \sum _{i\in Recursos}\sum _{j\in Aviones}\sum _{v\in Viajes}\sum _{k\in Zonas}VI_i\cdot x_{i,j,v,k}\cdot MI_k $$

* Calculo de costo total de transporte

$$ \sum _{j\in Aviones}CF_j\cdot z_j+\sum _{j\in Aviones}\:\sum _{v\in Viajes}\:\sum _{k\in Zonas}CK_j\cdot D_k\cdot y_{j,v,k} $$


### Restricciones

1. Disponibilidad total por recurso

$$ \sum _{j\in J}\sum _{v\in V}\sum _{k\in K}x_{i,j,v,k}\le D_i,\:\forall i\in I $$

2. Satisfacción mínima de demanda por zona

$$ \sum _{j\in J}\sum _{v\in V}x_{i,j,v,k}\ge N_{i,k},\:\forall i\in I,\:\forall k\in K $$

3. Capacidad de peso por avión y por viaje 

$$ \sum _{i\in I}P_i\cdot x_{i,j,v,k}\le CP_j\cdot y_{j,v,k},\:\forall j\in J,\:\forall v\in V,\:\forall k\in K $$

4. Capacidad de volumen por avión y por viaje 

$$ \sum _{i\in I}V_i\cdot x_{i,j,v,k}\le CV_j\cdot y_{j,v,k},\:\forall j\in J,\:\forall v\in V,\:\forall k\in K $$

5. Un avión visita como máximo una zona por viaje 

$$ \sum _{k\in K}y_{j,v,k}\le 1,\:\forall j\in J,\:\forall v\in V $$

6. Máximo 2 viajes por avión y coherencia con uso del avión

$$ \sum _{v\in V}\sum _{k\in K}y_{j,v,k}\le 2\cdot z_j\:\wedge \:z_j\le \sum _{v\in V}\sum _{k\in K}y_{j,v,k},\:\forall j\in J\: $$

7. Seguridad: Medicinas no pueden transportarse en el avión 1

$$ x_{Medicinas,1,v,k}=0 $$

8. Compatibilidad: Agua y Equipos Médicos no pueden viajar juntos

$$ p_{Agua\:potable,j,v}+p_{Equipos\:medicos,j,v}\le 1,\:\forall j\in J,\:\forall v\in V $$

9. Definición de presencia de un recurso en un viaje

$$ \sum _{k\in K}x_{i,j,v,k}\le D_i\cdot p_{i,j,v}\wedge \sum _{k\in K}x_{i,j,v,k}\ge p_{i,j,v},\:\forall i\in I,\:\forall j\in J,\:\forall v\in V $$

10. Consistencia entre envío y asignación de zona

$$ x_{i,j,v,k}\le D_i\cdot y_{j,v,k},\:\forall i\in I,\:\forall j\in J,\:\forall v\in V $$

### Elección del método de optimización multi-objetivo

Para resolver el problema multiobjetivo, vamos a usar el método ϵ-constraint, tomando como objetivo principal maximizar el impacto social generado por la entrega de recursos ($ Z₁ $) y convirtiendo el cálculo del costo total de transporte en una restricción ($ Z_2 < ϵ $). Escogimos este enfoque porque el modelo es un problema entero mixto, lo que produce una región factible no convexa, donde la suma ponderada puede dejar fuera soluciones ubicadas en porciones no convexas del frente de Pareto.

## 2. Implementación del Método ϵ-constraint 

In [18]:
from pyomo.environ import *
import math

# Creacion del modelo 
model = ConcreteModel()

# Recursos(Conjunto)
model.I = Set(initialize=["Alimentos_basicos", "Medicinas", "Equipos_medicos", "Agua_potable", "Mantas"])

# Aviones (Conjunto)
model.J = Set(initialize=[1, 2, 3, 4])

# Zonas destino (Conjunto)
model.K = Set(initialize=["A", "B", "C", "D"])

# Viajes posibles por avion (Conjunto)
model.V = Set(initialize=[0, 1, 2])

# Valor del impacto por recurso (Parametro)
Valor_impacto_recursos = {"Alimentos_basicos": 50, "Medicinas": 100, "Equipos_medicos": 120, "Agua_potable": 60, "Mantas": 40}  # USD / TON (original)

# Peso por unidad de recurso (Parametro)
Peso_recursos  = {"Alimentos_basicos": 5.0,  "Medicinas": 5.0,  "Equipos_medicos": 0.3, "Agua_potable": 6.0,  "Mantas": 3.0}   # TON / unidad

# Volumen por unidad de recurso (Parametro)
Volumen_recursos= {"Alimentos_basicos": 3.0,  "Medicinas": 1.0,  "Equipos_medicos": 0.5, "Agua_potable": 4.0,  "Mantas": 2.0}   # m3 / unidad

# Unidades disponibles de los recursos (Parametro)
Unidades_recursos = {"Alimentos_basicos": 12, "Medicinas": 15, "Equipos_medicos": 40, "Agua_potable": 15, "Mantas": 20}

# Capacidad de peso de los aviones (Parametro)
Capacidad_peso_aviones = {1: 40, 2: 50, 3: 60, 4: 45}            

# Capacidad de volumen de los aviones (Parametro)
Capacidad_volumen_aviones = {1: 35, 2: 40, 3: 45, 4: 38}       

# Costo fijo de los aviones (Parametro)
Costo_fijo_aviones = {1: 15, 2: 20, 3: 25, 4: 18}            

# Costo por km de los aviones (Parametro)
Costo_kilometro_aviones = {1: 0.020, 2: 0.025, 3: 0.030, 4: 0.022}

# Distancias a zonas (Parametro)
Distancia_zonas = {"A": 800, "B": 1200, "C": 1500, "D": 900}

# Multiplicador de impacto de cada zona (Parametro)
Multiplicador_impacto_zonas = {"A": 1.2, "B": 1.0, "C": 0.9, "D": 1.1}

# Necesidades de las zonas (Parametros)
Necesidades_zonas = {
    ("Alimentos_basicos","A"): 8, ("Agua_potable","A"): 6, ("Medicinas","A"): 2, ("Equipos_medicos","A"): 0.6, ("Mantas","A"): 3,
    ("Alimentos_basicos","B"):12, ("Agua_potable","B"): 9, ("Medicinas","B"): 3, ("Equipos_medicos","B"): 0.9, ("Mantas","B"): 5,
    ("Alimentos_basicos","C"):16, ("Agua_potable","C"):12, ("Medicinas","C"): 4, ("Equipos_medicos","C"): 1.2, ("Mantas","C"): 7,
    ("Alimentos_basicos","D"):10, ("Agua_potable","D"): 8, ("Medicinas","D"): 2, ("Equipos_medicos","D"): 0.6, ("Mantas","D"): 4
}

# Necesidades transformadas de toneladas a unidades
Necesidades_unidades = {}
for (i,k), ton in Necesidades_zonas.items():
    p = Peso_recursos[i]
    n_units = math.ceil(ton / p)
    Necesidades_unidades[(i,k)] = int(n_units)

# Parametros definidos

model.VI = Param(model.I, initialize=Valor_impacto_recursos)
model.P = Param(model.I, initialize=Peso_recursos)
model.VOL = Param(model.I, initialize=Volumen_recursos)
model.D = Param(model.I, initialize=Unidades_recursos)
model.CP = Param(model.J, initialize=Capacidad_peso_aviones)
model.CV = Param(model.J, initialize=Capacidad_volumen_aviones)
model.CF = Param(model.J, initialize=Costo_fijo_aviones)
model.CK = Param(model.J, initialize=Costo_kilometro_aviones)
model.DZ = Param(model.K, initialize=Distancia_zonas)
model.MI = Param(model.K, initialize=Multiplicador_impacto_zonas)
model.N = Param(model.I, model.K, initialize=Necesidades_unidades)

# Variables definidas

model.x = Var(model.I, model.J, model.V, model.K, within=NonNegativeIntegers)
model.y = Var(model.J, model.V, model.K, within=Binary)
model.z = Var(model.J, within=Binary)
model.p = Var(model.I, model.J, model.V, within=Binary)



# Funcion objetivo principal (Z_1)

def funcion_objetivo_impacto_social(model):
    return sum(model.VI[i] * model.x[i,j,v,k] * model.MI[k] for i in model.I for j in model.J for v in model.V for k in model.K)
model.Impacto = Objective(rule = funcion_objetivo_impacto_social, sense=maximize)



# Disponibilidad total (Restriccion)
def disponibilidad(model, i):
    return sum(model.x[i,j,v,k] for j in model.J for v in model.V for k in model.K) <= model.D[i]
model.R2_disp = Constraint(model.I, rule=disponibilidad)

# Satisfacción minima por zona (Restriccion)
def demanda_min(model, i, k):
    return sum(model.x[i,j,v,k] for j in model.J for v in model.V) >= model.N[i,k]
model.R3_demanda = Constraint(model.I, model.K, rule=demanda_min)

# Capacidad de peso (Restriccion)
def cap_peso(model, j, v, k):
    return sum(model.x[i,j,v,k] * model.P[i] for i in model.I) <= model.CP[j] * model.y[j,v,k]
model.R4_cap_peso = Constraint(model.J, model.V, model.K, rule=cap_peso)

# Capacidad de volumen (Restriccion)
def cap_vol(model, j, v, k):
    return sum(model.x[i,j,v,k] * model.VOL[i] for i in model.I) <= model.CV[j] * model.y[j,v,k]
model.R5_cap_vol = Constraint(model.J, model.V, model.K, rule=cap_vol)

# Un avion no puede visitar mas de una zona por viaje (Restriccion)
def una_zona(model, j, v):
    return sum(model.y[j,v,k] for k in model.K) <= 1
model.R6_una_zona = Constraint(model.J, model.V, rule=una_zona)

# Maximo 2 viajes por avion (Restriccion)
def max_viajes(model, j):
    return sum(model.y[j,v,k] for v in model.V for k in model.K) <= 2 * model.z[j]
model.R7_max_viajes = Constraint(model.J, rule=max_viajes)

def coherencia_z(model, j):
    return model.z[j] <= sum(model.y[j,v,k] for v in model.V for k in model.K)
model.R7_coherencia = Constraint(model.J, rule=coherencia_z)

# Medicinas no pueden ir en avion 1 (Restriccion)
def no_meds_plane1(model, v, k):
    return model.x["Medicinas", 1, v, k] == 0
model.R8_no_meds = Constraint(model.V, model.K, rule=no_meds_plane1)

# Agua y equipos medcos no pueden ir juntos (Restriccion)
def incompatibilidad(model, j, v):
    return model.p["Agua_potable", j, v] + model.p["Equipos_medicos", j, v] <= 1
model.R9_incomp = Constraint(model.J, model.V, rule=incompatibilidad)

# 10. Persistencia del recurso (Restriccion)
def presencia_up(model, i, j, v):
    return sum(model.x[i,j,v,k] for k in model.K) <= model.D[i] * model.p[i,j,v]
model.R10_presencia_up = Constraint(model.I, model.J, model.V, rule=presencia_up)

def presencia_low(model, i, j, v):
    return sum(model.x[i,j,v,k] for k in model.K) >= model.p[i,j,v]
model.R10_presencia_low = Constraint(model.I, model.J, model.V, rule=presencia_low)

# No se puede enviar recurso si no se asigno viaje y zona (Restriccion)
def envio_consistente(model, i, j, v, k):
    return model.x[i,j,v,k] <= model.CP[j] * model.y[j,v,k]
model.R11_envio_ok = Constraint(model.I, model.J, model.V, model.K, rule=envio_consistente)



# Funcion segundaria (Z_2)
def funcion_segundaria_costo_transporte(model):
    return sum(model.CF[j] * model.z[j] for j in model.J) + sum(model.CK[j] * model.DZ[k] * model.y[j,v,k] for j in model.J for v in model.V for k in model.K)
model.Costo = Expression(rule = funcion_segundaria_costo_transporte)

# Epsilon (Restriccion)
model.epsilon = Param(initialize=1e9, mutable=True)

def restriccion_epsilon(model):
    return model.Costo <= model.epsilon
model.R_eps = Constraint(rule=restriccion_epsilon)

solver = SolverFactory("gurobi")

In [19]:
result_impacto = solver.solve(model)
impacto_max = value(model.Impacto)
print("Impacto maximo de los recursos:", impacto_max)

model.Impacto.deactivate()
if hasattr(model, 'Costo_min'):
    model.del_component(model.Costo_min)

model.Costo_min = Objective(expr=model.Costo, sense=minimize)

result_costo = solver.solve(model)
costo_min = value(model.Costo)
print("Costo minimo del transporte:", costo_min)
model.Costo_min.deactivate()
model.Impacto.activate()

print(" ")
print(f"Valor optimo del impacto social: {value(model.Impacto):.2f}")
print(f"Valor del costo del transporte: {value(model.Costo):.2f}")
print("Detalle de cada avion:")

for j in model.J:

    if value(model.z[j]) < 0.5:
        print(f"Avion {j} -> No utilizado.\n")
        continue

    print(f"\n===== Avion {j} =====")

    for v in model.V:

        zona_asignada = None
        for k in model.K:
            if value(model.y[j, v, k]) > 0.5:
                zona_asignada = k
                break

        if zona_asignada is None:
            continue  

        print(f"\n Viaje {v} hacia zona {zona_asignada}")
        peso_usado_v = 0
        vol_usado_v = 0
        impacto_v = 0
        recursos_v = []

        for i in model.I:
            x_val = value(model.x[i, j, v, zona_asignada])

            if x_val > 1e-6:
                recursos_v.append((i, x_val))
                peso_usado_v += x_val * model.P[i]
                vol_usado_v += x_val * model.VOL[i]
                impacto_v += x_val * model.VI[i] * model.MI[zona_asignada]

        if len(recursos_v) == 0:
            print("No se transportaron recursos")
            continue

        print("Recursos transportados:")
        for (i, qty) in recursos_v:
            print(f"- {i}: {qty:.2f} unidades")

        print(f"Peso usado: {peso_usado_v:.2f} / {model.CP[j]}")
        print(f"Volumen usado: {vol_usado_v:.2f} / {model.CV[j]}")
        print(f"Impacto generado en este viaje: {impacto_v:.2f}")


Impacto maximo de los recursos: 9723.0
Costo minimo del transporte: 284.4
 
Valor optimo del impacto social: 3080.00
Valor del costo del transporte: 284.40
Detalle de cada avion:

===== Avion 1 =====

 Viaje 0 hacia zona C
Recursos transportados:
- Alimentos_basicos: 4.00 unidades
- Agua_potable: 2.00 unidades
Peso usado: 32.00 / 40
Volumen usado: 20.00 / 35
Impacto generado en este viaje: 288.00

 Viaje 1 hacia zona B
Recursos transportados:
- Alimentos_basicos: 3.00 unidades
- Equipos_medicos: 3.00 unidades
- Mantas: 2.00 unidades
Peso usado: 21.90 / 40
Volumen usado: 14.50 / 35
Impacto generado en este viaje: 590.00

===== Avion 2 =====

 Viaje 0 hacia zona D
Recursos transportados:
- Alimentos_basicos: 2.00 unidades
- Medicinas: 1.00 unidades
- Equipos_medicos: 2.00 unidades
Peso usado: 15.60 / 50
Volumen usado: 8.00 / 40
Impacto generado en este viaje: 484.00

 Viaje 1 hacia zona D
Recursos transportados:
- Agua_potable: 2.00 unidades
- Mantas: 2.00 unidades
Peso usado: 18.00 / 50